# This code will run all the functions from cleaning the data, processing it and then outputting the results

The directories may need to be adjusted as the code does not output all at once
- Due to short timeframes the code may not run as efficently. I have inputted this code from the various files I used throughout, so code may repeat itself.
- These were individual scripts that I used aggregated into this notebook
- There may be conficts within the code that might need to be resolved 

First we pull the main functions into python

In [ ]:
import pandas as pd
import yfinance as yf 
import numpy as np
#Need to add for different tickers

#ticker_list = ['QQQ', 'AMC', 'MSFT', 'ARKK', 'GME', 'SPY'] # AMZN
ticker_list = ['NVDA', 'XOM', 'AAPL', 'NFLX', 'TSLA', 'AMD','QQQ', 'MSFT', 'ARKK', 'GME', 'SPY', 'AMZN']
#df = pd.read_csv('../spy_2020.csv', index_col=0)


def merge_df(df, data):

    df['date']= pd.to_datetime(df['date'], format='%Y%m%d', errors='ignore')


    #resets the index, you cant just change names of indexes
    data.reset_index(inplace=True)
    data=data.rename(columns= {'Date': 'date'})

    data['Close'] = data['Close'].round(2) #rounding the price on close
    data = data[['date','Close', 'Adj Close']] #Pull only the date and the close 
    df = df.merge(data, on='date') #merge data onto the df using date

    #df['Close'].loc[((df['date'] == 'C')] 
    return df


#function to get date difference between epiry and the current date
def get_t(df_loc):
    exp_dates_convert = pd.to_datetime(df_loc['exdate'], format='%Y%M%d', errors='ignore')
    current_date_convert = pd.to_datetime(df_loc['date'],format='%Y-%M-%d', errors='ignore')
    days_to_expiry = exp_dates_convert - current_date_convert
    return (days_to_expiry.dt.days/365)



from datetime import datetime as dt
import time

    

We clean the data - much of the values were deep ITM options that weren't traded anymore

In [ ]:
 


start = time.time()

for i in ticker_list:
    loop_time = time.time()
    df = pd.read_csv(f'../{i}.csv')
    df['Intrinsic_Value'] = np.nan
    df['strike_price'] = df['strike_price']/1000
    
    final_day=df.date.head(1).item()
    final_day = str(final_day)
    date_object = dt.strptime(final_day, '%Y%M%d')
    final_day= dt.strftime(date_object, '%Y-%M-%d')

    data = yf.download(f"{i}", start=final_day, end="2021-01-01")
    df = merge_df(df, data)

    df['Intrinsic_Value'].loc[df['cp_flag'] == 'C'] = (df['Close']-df['strike_price']) 
    df['Intrinsic_Value'].loc[df['cp_flag'] == 'P'] = (df['strike_price']-df['Close']) 



    df['impl_volatility'].loc[(df['impl_volatility'].isnull())] = 0
    
    df['delta'].loc[df['delta'].isnull()] = 1
    df['gamma'].loc[df['gamma'].isnull()] = 0

    #if the gamma is NaN, if sigma=0 as the entire equation is divided by it then we just set it to 0

    df['delta'].loc[(df['delta']) == 0 & (df['cp_flag'] == 'P')] = 1



    df['theta'].loc[(df['theta'].isnull()) & (df['cp_flag'] == 'C')] = 0
    df['theta'].loc[df['theta'].isnull() & df['cp_flag'] == 'P' ] = 0


    df['vega'].loc[df['vega'].isnull()] = 0

    df.to_csv(f'{i}_edit_2.csv')

    del df
    loop_time_end = time.time()
    print(loop_time-loop_time_end)

end = time.time()

print(f'final time is {end - start}')

We add the higer order greeks

In [ ]:
import pandas as pd
import numpy as np 
#from math import sqrt, pi,log, e
from enum import Enum
import scipy.stats as stat
from scipy.stats import norm


def get_t(df):
    exp_dates_convert = pd.to_datetime(df['exdate'], format='%Y%m%d', errors='ignore')

    current_date_convert = pd.to_datetime(df['date'],format='%d/%M/%Y', errors='ignore')

    print(exp_dates_convert.dtypes)
    print(current_date_convert.dtypes)

    print(exp_dates_convert)
    print(current_date_convert)
    exp_dates_convert  = exp_dates_convert.astype('datetime64[ns]')
    current_date_convert  = current_date_convert.astype('datetime64[ns]')
    days_to_expiry = exp_dates_convert - current_date_convert
    return (days_to_expiry.dt.days/365)



def zomma():
    d1 = (np.log(S/K)+(r+v*v/2.)*t)/(v*np.sqrt(t))
    d1 = np.nan_to_num(d1)
    d2 = d1-v*np.sqrt(t)
    zomma = np.exp(t) * (norm.pdf(d1)*(d1*d2-1)) / (S * v**2 * np.sqrt(t))
    return zomma

def vanna():
    d1 = 1 / (v * np.sqrt(t)) * ( np.log(S/K) + (r + v**2/2) * t)
    d2 = d1 - v * np.sqrt(t)
    vanna = -np.exp(t)* (norm.pdf(d1)*((d2)/v))
    return vanna


ticker_list = ['NVDA', 'XOM', 'AAPL', 'NFLX', 'TSLA', 'AMD','QQQ', 'MSFT', 'ARKK', 'GME', 'SPY', 'AMZN']
for i in ticker_list:
    
    df = pd.read_csv(f'../{i}_edit_2.csv')
    S = df['Close']
    r=0.1
    q=0
    K= df['strike_price']
    v = df['impl_volatility']
    t = get_t(df)
    print(zomma())

    df['zomma'] = zomma()
    df['vanna'] = vanna()

    df.to_csv(f'../{i}_added_greeks.csv')
    del df

We calculate the assoicated models

In [ ]:
import pandas as pd 
import numpy as np
from datetime import date, timedelta
import matplotlib.pyplot as plt 
from datetime import datetime as dt
import yfinance as yf
import time 



def merge_df(df, data):

    df['date']= pd.to_datetime(df['date'], format='%Y-%m-%d', errors='ignore')
    #print(df.date[1])

    #resets the index, you cant just change names of indexes
    data.reset_index(inplace=True)
    data=data.rename(columns= {'Date': 'date'})
    
    data['underling_volume'] = data['Volume'].round(2) #rounding the price on close
    data = data[['date','underling_volume']]
    
     #Pull only the date and the close 
    df = df.merge(data, on='date') #merge data onto the df using date
    return df


ticker_list = ['NVDA', 'XOM', 'AAPL', 'NFLX', 'TSLA', 'AMD','QQQ', 'MSFT', 'ARKK', 'GME', 'SPY', 'AMZN']
for i in ticker_list:

    
        

    new_df = pd.read_csv(f'../{i}_added_greeks.csv', usecols=['date', 'cp_flag',  'delta', 'gamma', 'volume', 'open_interest', 'impl_volatility', 'vanna', 'zomma'])
    new_df = new_df.loc[(new_df['volume'].notnull()) & (new_df['open_interest']>0) & (new_df['volume']>0)]
    new_df = new_df.fillna(0)
    print(new_df)

    final_day=new_df.date.head(1).item()
    df_len = len(pd.date_range(final_day, end='1-1-2021', freq= 'B'))


    print(final_day)
    final_day = dt.strptime(final_day, '%Y-%m-%d')
    day_delta=[]
    gex_day =[]
    date_list=[]
    call_volume_aggregate_list=[]
    put_volume_aggregate_list = []
    call_open_interest_aggregate_list = []
    put_open_interest_aggregate_list = []
    call_iv_aggregate_list = []
    put_iv_aggregate_list=[]
    nope_oi_list =[]
    gex_volume_list=[]
    net_vanna_oi_list = []
    net_vanna_volume_list =[]
    net_zomma_oi_list =[]
    net_zomma_volume_list =[] 
    day_delta_dict = {}


    data = yf.download(f"{i}", start=final_day, end="2021-01-01")

    snp_return = data['Close'] - data['Close'].shift()

    snp_return_percentage = ((data['Close'] - data['Close'].shift())/data['Close'].shift())*100 
    print(data['Close'])
    print(snp_return)
    print(snp_return_percentage)

    k=0
    printcounter = 0
    n_time = time.time()
    time_holder = time.time()
    #d = {'date' : np.nan, 'delta': np.nan, 'NOPE': np.nan}
    df_test = pd.DataFrame(columns= ['date', 'delta'])
    


    #EDIT THIS SO THAT IT CHANGES TO A VECTOR SOLUTION AND NOT A LOOP, maybe look at iterrows 
    for j in pd.date_range(final_day, end='1-1-2021', freq='B'):
        

        #print(j)
        df_loc = new_df[(new_df['cp_flag']=='C') & (new_df['date']==j)] # ==j =new_df['date'][k])]
        df_put = new_df[(new_df['cp_flag']=='P') & (new_df['date']==j)] # MAYBE ADD IN WHERE VOLUME IS BIGGER THAN 0 AS WE DONT LOOK AT THAT FOR NOW 

        df_test['date'] = j  


        nope = ((df_loc['volume']*df_loc['delta']).sum()+(df_put['volume']*df_put['delta']).sum())   
        nope_oi =  ((df_loc['open_interest']*df_loc['delta']).sum()+(df_put['open_interest']*df_put['delta']).sum()) 


        gex = ((df_loc['open_interest']*df_loc['gamma']).sum()-(df_put['open_interest']*df_put['gamma']).sum())
        gex_volume = ((df_loc['volume']*df_loc['gamma']).sum()-(df_put['volume']*df_put['gamma']).sum())


        net_vanna_oi =  ((df_loc['open_interest']*df_loc['vanna']).sum()+(df_put['open_interest']*df_put['vanna']).sum())
        net_vanna_volume =  ((df_loc['volume']*df_loc['vanna']).sum()+(df_put['volume']*df_put['vanna']).sum())

        net_zomma_oi =  ((df_loc['open_interest']*df_loc['zomma']).sum()+(df_put['open_interest']*df_put['zomma']).sum())
        net_zomma_volume =  ((df_loc['volume']*df_loc['zomma']).sum()+(df_put['volume']*df_put['zomma']).sum())


        call_volume_aggregate = df_loc['volume'].sum()
        put_volume_aggregate = df_put['volume'].sum()

        call_open_interest_aggregate = df_loc['open_interest'].sum()
        put_open_interest_aggregate = df_put['open_interest'].sum()


        call_iv_aggregate = (df_loc['open_interest']*df_loc['impl_volatility']).mean()
        put_iv_aggregate = (df_put['open_interest']*df_put['impl_volatility']).mean()
        delta_holder = nope


        gex_day.append(gex)
        day_delta.append(delta_holder)
        date_list.append(j)
        call_volume_aggregate_list.append(call_volume_aggregate)
        put_volume_aggregate_list.append(put_volume_aggregate)
        call_open_interest_aggregate_list.append(call_open_interest_aggregate)
        put_open_interest_aggregate_list.append(put_open_interest_aggregate)
        call_iv_aggregate_list.append(call_iv_aggregate)
        put_iv_aggregate_list.append(put_iv_aggregate)

        nope_oi_list.append(nope_oi)
        gex_volume_list.append(gex_volume)
        net_vanna_oi_list.append(net_vanna_oi)
        net_vanna_volume_list.append(net_vanna_volume)
        net_zomma_oi_list.append(net_zomma_oi)
        net_zomma_volume_list.append(net_zomma_volume)



        if (k % 100 == 0) and k>0:
            
            old_time = time.time()

            print(f'number {k}'); print(f'{(old_time - n_time)/60} mins total'); print(f'{round((k/df_len)*100, 3)}%')
            print(df_len, k)
            print((df_len-k)/100)
            print(f'estimated time : {((old_time-time_holder)*(df_len-k)/100)/60} mins') #df_len/k)-k

            print(f'{k} out of {df_len} days')
            print('========='*6)

            time_holder = time.time()



        del df_loc
        del df_put
        k=k+1

    day_delta_dict['delta'] = day_delta
    day_delta_dict['gex'] = gex_day
    day_delta_dict['date'] = date_list
    day_delta_dict['call_volume_aggregate_list'] = call_volume_aggregate_list
    day_delta_dict['put_volume_aggregate_list'] = put_volume_aggregate_list
    day_delta_dict['call_open_interest_aggregate'] = call_open_interest_aggregate_list
    day_delta_dict['put_open_interest_aggregate'] = put_open_interest_aggregate_list
    day_delta_dict['call_iv_aggregate'] = call_iv_aggregate_list
    day_delta_dict['put_iv_aggregate']= put_iv_aggregate_list

    day_delta_dict['nope_oi']= nope_oi_list
    day_delta_dict['gex_volume']= gex_volume_list
    day_delta_dict['net_vanna_oi']= net_vanna_oi_list
    day_delta_dict['net_vanna_volume']= net_vanna_volume_list
    day_delta_dict['net_zomma_oi']= net_zomma_oi_list
    day_delta_dict['net_zomma_volume']= net_zomma_volume_list

    df_final = pd.DataFrame(day_delta_dict)
    
    df_final = merge_df(df_final, data)


    df_final['nope'] = df_final['delta'] / df_final['underling_volume'] 

    df_final['nope_oi_normalised'] = df_final['nope_oi'] / df_final['underling_volume'] 
    df_final['gex_volume_normalised'] = df_final['gex_volume'] / df_final['underling_volume']
    df_final['gex_normalised'] = df_final['gex'] / df_final['underling_volume'] 
    df_final['net_vanna_oi_normalised']= df_final['net_vanna_oi'] / df_final['underling_volume'] 
    df_final['net_vanna_volume_normalised']= df_final['net_vanna_volume'] / df_final['underling_volume'] 
    df_final['net_zomma_oi_normalised']= df_final['net_zomma_oi']/ df_final['underling_volume'] 
    df_final['net_zomma_volume_normalised']= df_final['net_zomma_volume']/ df_final['underling_volume'] 

    print('=================')
    print(df_final)
    print('=================')


    df_final.to_csv(f'../{i}nope_3.csv')
    day_delta.clear()
    gex_day.clear()
    call_volume_aggregate_list.clear()
    put_volume_aggregate_list.clear()
    call_open_interest_aggregate_list.clear()
    put_open_interest_aggregate_list.clear() 
    call_iv_aggregate_list.clear()
    put_iv_aggregate_list.clear()

    nope_oi_list.clear()
    gex_volume_list.clear()
    net_vanna_oi_list.clear()
    net_vanna_volume_list.clear()
    net_zomma_oi_list.clear()
    net_zomma_volume_list.clear()

    del df_final
    del day_delta_dict

We get the regression values

In [ ]:
import pandas as pd 
import yfinance as yf
from datetime import datetime as dt
import statsmodels.api as sm
import numpy as np

def merge_df(df, data):

    df['date']= pd.to_datetime(df['date'], format='%Y-%m-%d', errors='ignore')
    #print(df.date[1])

    #resets the index, you cant just change names of indexes
    data.reset_index(inplace=True)
    data=data.rename(columns= {'Date': 'date'})
    
    data['close'] = data['Close'].round(2) #rounding the price on close
    data = data[['date','close', 'daily_return', 'stock_return_percentage']]
    
     #Pull only the date and the close 
    df = df.merge(data, on='date') #merge data onto the df using date

    #df['Close'].loc[((df['date'] == 'C')] 
    return df



tickers = ['MSFT', 'NVDA', 'XOM', 'AAPL', 'NFLX', 'TSLA', 'AMD']

#These are the additional regressors tested, I only included the main ones, however some of the models showed promising results, as I alluded to in the main dissertation
#regresors = ['nope', 'gex', 'gex_positive', 'gex_negative', 'nope_oi', 'nope_oi_normalised', 'gex_volume_normalised', 'gex_normalised', 'net_vanna_oi',	'net_vanna_volume',	'net_zomma_oi',	'net_zomma_volume',	'net_vanna_oi_normalised',	'net_vanna_volume_normalised',	'net_zomma_oi_normalised', 'net_zomma_volume_normalised']


regresors = ['nope', 'gex', 'gex_positive', 'gex_negative', 'nope_oi','net_vanna_oi','net_zomma_oi']



regression_dict = {}
regression_dict['regresors'] = regresors
paramvalues_list = []
p_values_list = []
r_squared_list =[]
intercept_list=[]

for ticker in tickers:
  #  print(ticker)
    df = pd.read_csv(f'../{ticker}nope_3.csv')
    
# print(df)
    data = yf.download(ticker)
    
    dividor = 100000
    df['nope'] = df['nope']
    df['gex'] = df['gex']/dividor

    df['nope_oi'] = df['nope_oi']/dividor
    df['net_vanna_oi'] = df['net_vanna_oi']/dividor
    df['net_zomma_oi'] = df['net_zomma_oi']/dividor

    df['nope'] = df['nope']*100
    df['gex'] = df['gex']*100
    df['gex_positive'] = df['gex'].loc[(df['gex']>0)]
    df['gex_negative'] = df['gex'].loc[(df['gex']<0)]
    final_day=df.date.head(1).item()
    final_day = dt.strptime(final_day, '%Y-%m-%d')

    data = yf.download(ticker, start=final_day, end="2021-01-01")

    stock_return = data['Close'] - data['Close'].shift()

    data['daily_return'] = stock_return
    stock_return_percentage = ((data['Close'] - data['Close'].shift())/data['Close'].shift())*100
    data['stock_return_percentage'] = stock_return_percentage


    df = merge_df(df, data)


    


    
    import statsmodels.api as sm

    df = df.set_index('date')
    start_year = 2015
    end_year = 2020
    df = df[f'{start_year}0101':f'{end_year}1230']
    df = df.fillna(0)
    df = df.replace([np.inf, -np.inf], 0)


    for j in regresors:


        regressor_var = j
    
    
        
       
        #print(ticker)
    # df = df.replace([np.inf, -np.inf], 0, inplace=True)
      #  print(df[j])
        X = df[j]
        y = df.stock_return_percentage.shift(fill_value=0)

        

        #X = np.array(X)
        #y = np.array(y)
        y= list(y)
        X= list(X)
        X = sm.add_constant(X)
        #print(pd.isNan(X))
        #print(y)
        #X = sm.add_constant(X)
        ols_resid = sm.OLS(y, X).fit().resid
        res_fit = sm.OLS(ols_resid[1:], ols_resid[:-1]).fit()
        rho = res_fit.params
        
        from scipy.linalg import toeplitz

        
        order = toeplitz(range(len(ols_resid)))
        sigma = rho**order
        gls_model = sm.GLS(y, X, sigma=sigma)
        gls_results = gls_model.fit()
        print(j)
        print(gls_results.summary())

        #gls_results = sm.OLS(y, X).fit()
        #print(gls_results.summary())
        #print(gls_results.params)
       # print(gls_results.rsquared_adj)
        #print(gls_results.pvalues)
        gls_results.params[0] = gls_results.params[0]*100
        gls_results.params[0] = round(gls_results.params[0], 5) 
        gls_results.params[1] = round(gls_results.params[1], 5) 
        gls_results.pvalues[0] = round(gls_results.pvalues[0], 5)
        param_result_string = str(gls_results.params)#[0])
        #gls_rsquared = gls_results.rsquared[0]

        if gls_results.pvalues[0] < 0.1 and gls_results.pvalues[0] > 0.05:
            new_intercept_val = f'{str(gls_results.params[0])}*  ({round(gls_results.tvalues[0], 3)})'
        if gls_results.pvalues[0] < 0.05 and gls_results.pvalues[0] > 0.01:
            new_intercept_val = f'{str(gls_results.params[0])}**  ({round(gls_results.tvalues[0], 3)})'
        if gls_results.pvalues[0] < 0.01:
            new_intercept_val = f'{str(gls_results.params[0])}***  ({round(gls_results.tvalues[0], 3)})'
        else: 
            new_intercept_val = f'{str(gls_results.params[0])} ({round(gls_results.tvalues[0], 3)})'

        if gls_results.pvalues[1] < 0.1 and gls_results.pvalues[1] > 0.05:
            new_beta_val = f'{str(gls_results.params[1])}e-5*  ({round(gls_results.tvalues[1], 3)})'
        if gls_results.pvalues[1] < 0.05 and gls_results.pvalues[1] > 0.01:
            new_beta_val = f'{str(gls_results.params[1])}e-5**  ({round(gls_results.tvalues[1], 3)})'
        if gls_results.pvalues[1] < 0.01:
            new_beta_val = f'{str(gls_results.params[1])}e-5*** ({round(gls_results.tvalues[1], 3)})'
        else: 
            new_beta_val = f'{str(gls_results.params[1])}e-5 ({round(gls_results.tvalues[1], 3)})'

        '''
        if 'Regressor' in regression_dict:
            regression_dict['regressor'].append(regressor_var)
        else:
            regression_dict['regressor'] = regressor_var
        '''

        #if f'{ticker}_pararm' in regression_dict:
        param_result_string = str(gls_results.params[0])
        #print(param_result_string)
        intercept_list.append(new_intercept_val)
        paramvalues_list.append(new_beta_val)
        r_squared_list.append(round(gls_results.rsquared_adj*100, 3))

        #p_values_list.append(new_p_val)

    param_holder = paramvalues_list
    p_holder = p_values_list

    r_holder = r_squared_list

    regression_dict[f'{ticker} Intercept'] = intercept_list
    regression_dict[f'{ticker} beta t'] = param_holder
    regression_dict[f'{ticker} R Squared %'] = r_holder
        #change this to the code on top after
   # regression_dict[f'{ticker}_pvalues'] = p_holder
    
    '''
    paramvalues_list.clear()
    p_values_list.clear()

    '''
    #print(f"{ticker}, {len(regression_dict[f'{ticker}_pararm'])}")
        #store them in a list then a dict, then save them to csv, then add them into word or whatever
        #add multiple values


    del df
  #  print(regression_dict)
    #del paramvalues_list
    #del p_values_list
    
   # print(len(paramvalues_list))
   # print(len(intercept_list))
  #  print(len(param_holder[0]))
   # print(len(param_holder[1]))
   # print(len(p_values_list))
   # print(len(r_squared_list))
    intercept_list =[]
    paramvalues_list = []
    p_values_list = []
    r_squared_list = []

    
print(regression_dict)
print(len(regression_dict))
df_final = pd.DataFrame(regression_dict)
df_final.to_csv('single_stock_regression_3.csv')



For GME

In [ ]:
import pandas as pd 
import yfinance as yf
from datetime import datetime as dt
import statsmodels.api as sm
import numpy as np

def merge_df(df, data):

    df['date']= pd.to_datetime(df['date'], format='%Y-%m-%d', errors='ignore')
    #print(df.date[1])

    #resets the index, you cant just change names of indexes
    data.reset_index(inplace=True)
    data=data.rename(columns= {'Date': 'date'})
    
    data['close'] = data['Close'].round(2) #rounding the price on close
    data = data[['date','close', 'daily_return', 'stock_return_percentage']]
    
     #Pull only the date and the close 
    df = df.merge(data, on='date') #merge data onto the df using date
    return df



tickers = ['GME']
#regresors = ['nope', 'gex', 'gex_positive', 'gex_negative', 'nope_oi', 'nope_oi_normalised', 'gex_volume_normalised', 'gex_normalised', 'net_vanna_oi',	'net_vanna_volume',	'net_zomma_oi',	'net_zomma_volume',	'net_vanna_oi_normalised',	'net_vanna_volume_normalised',	'net_zomma_oi_normalised', 'net_zomma_volume_normalised']
regresors = ['nope', 'gex', 'gex_positive', 'gex_negative', 'nope_oi','net_vanna_oi','net_zomma_oi']



regression_dict = {}
regression_dict['regresors'] = regresors
paramvalues_list = []
p_values_list = []
r_squared_list =[]
intercept_list=[]

for ticker in tickers:
    df = pd.read_csv(f'../{ticker}nope_3.csv')

    data = yf.download(ticker)
    
    
    dividor = 100000
    df['nope'] = df['nope']
    df['gex'] = df['gex']/dividor

    df['nope_oi'] = df['nope_oi']/dividor
    df['net_vanna_oi'] = df['net_vanna_oi']/dividor
    df['net_zomma_oi'] = df['net_zomma_oi']/dividor
    df['gex_positive'] = df['gex'].loc[(df['gex']>0)]
    df['gex_negative'] = df['gex'].loc[(df['gex']<0)]
    final_day=df.date.head(1).item()
    final_day = dt.strptime(final_day, '%Y-%m-%d')

    data = yf.download(ticker, start=final_day, end="2021-01-01")

    stock_return = data['Close'] - data['Close'].shift()

    data['daily_return'] = stock_return
    stock_return_percentage = ((data['Close'] - data['Close'].shift())/data['Close'].shift())*100
    data['stock_return_percentage'] = stock_return_percentage


    df = merge_df(df, data)


    


    
    import statsmodels.api as sm

#nope_oi_normalised	gex_volume_normalised	gex_normalised	net_vanna_oi_normalised	net_vanna_volume_normalised	net_zomma_oi_normalised	net_zomma_volume_normalised
    df = df.set_index('date')
    start_year = 2020
    end_year = 2020
    df = df[f'{start_year}0101':f'{end_year}1230']
    df = df.fillna(0)
    df = df.replace([np.inf, -np.inf], 0)


    for j in regresors:


        regressor_var = j
        X = df[j]
        y = df.stock_return_percentage.shift(fill_value=0)

        y= list(y)
        X= list(X)
        X = sm.add_constant(X)

        ols_resid = sm.OLS(y, X).fit().resid
        res_fit = sm.OLS(ols_resid[1:], ols_resid[:-1]).fit()
        rho = res_fit.params
        
        from scipy.linalg import toeplitz

        
        order = toeplitz(range(len(ols_resid)))
        sigma = rho**order
        gls_model = sm.GLS(y, X, sigma=sigma)
        gls_results = gls_model.fit()
        print(gls_results.rsquared_adj)
        #print(gls_results.pvalues)
        gls_results.params[0] = gls_results.params[0]*100
        gls_results.params[0] = round(gls_results.params[0], 5) 
        gls_results.pvalues[0] = round(gls_results.pvalues[0], 5)
        param_result_string = str(gls_results.params)#[0])
        #gls_rsquared = gls_results.rsquared[0]

        if gls_results.pvalues[0] < 0.1 and gls_results.pvalues[0] > 0.05:
            new_intercept_val = f'{str(gls_results.params[0])}*  ({round(gls_results.tvalues[0], 3)})'
        if gls_results.pvalues[0] < 0.05 and gls_results.pvalues[0] > 0.01:
            new_intercept_val = f'{str(gls_results.params[0])}**  ({round(gls_results.tvalues[0], 3)})'
        if gls_results.pvalues[0] < 0.01:
            new_intercept_val = f'{str(gls_results.params[0])}***  ({round(gls_results.tvalues[0], 3)})'
        else: 
            new_intercept_val = f'{str(gls_results.params[0])} ({round(gls_results.tvalues[0], 3)})'

        if gls_results.pvalues[1] < 0.1 and gls_results.pvalues[1] > 0.05:
            new_beta_val = f'{str(gls_results.params[1])}e-5*  ({round(gls_results.tvalues[1], 3)})'
        if gls_results.pvalues[1] < 0.05 and gls_results.pvalues[1] > 0.01:
            new_beta_val = f'{str(gls_results.params[1])}e-5**  ({round(gls_results.tvalues[1], 3)})'
        if gls_results.pvalues[1] < 0.01:
            new_beta_val = f'{str(gls_results.params[1])}e-5*** ({round(gls_results.tvalues[1], 3)})'
        else: 
            new_beta_val = f'{str(gls_results.params[1])}e-5 ({round(gls_results.tvalues[1], 3)})'

        '''
        if 'Regressor' in regression_dict:
            regression_dict['regressor'].append(regressor_var)
        else:
            regression_dict['regressor'] = regressor_var
        '''

        #if f'{ticker}_pararm' in regression_dict:
        param_result_string = str(gls_results.params[0])
        #print(param_result_string)
        intercept_list.append(new_intercept_val)
        paramvalues_list.append(new_beta_val)
        r_squared_list.append(round(gls_results.rsquared_adj*100, 3))

        #p_values_list.append(new_p_val)

    param_holder = paramvalues_list
    p_holder = p_values_list

    r_holder = r_squared_list

    regression_dict[f'{ticker} Intercept'] = intercept_list
    regression_dict[f'{ticker} beta t'] = param_holder
    regression_dict[f'{ticker} R Squared %'] = r_holder


    del df
    print(regression_dict)
    intercept_list =[]
    paramvalues_list = []
    p_values_list = []
    r_squared_list = []

    
print(regression_dict)
print(len(regression_dict))
df_final = pd.DataFrame(regression_dict)
df_final.to_csv('GME_regression_values_3.csv')

